# 8A2_common_loops_genes

7/7/2021

what are the genes touching the common loops and are those genes all expressed

In [6]:
import pandas as pd
import os, glob
import pybedtools
import numpy as np

In [7]:
save_dir = '../data/processed/fig4_modelling/common_genes'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# get commonly expressed genes

In [8]:
#get rna
normal_tissues = ['Airway','Astrocytes','Bladder','Colon','Esophageal','GDSD6','GM12878','HMEC','Melanocytes','Ovarian',
'Pancreas','Prostate','Renal','Thyroid','Uterine']
normal_tissues_dict = dict(zip(normal_tissues,range(len(normal_tissues))))
rna_df = pd.read_csv('../data/interim/rna/tissue_tpm_sym.csv',index_col=0)
rna_df_norm = rna_df[normal_tissues]

In [9]:
THRES=1

In [35]:
# common genes
print(sum((rna_df_norm>THRES).sum(axis=1)==15), 'number of common genes')
print(sum((rna_df_norm>THRES).sum(axis=1)==1), 'number of unique genes')
print(rna_df_norm.shape)
common_genes = []
# counter = 0 
for gene, row in pd.DataFrame(rna_df_norm>THRES).iterrows():
#     if counter>5:
#         break
#     counter +=1
    if np.array(row).all():
        common_genes.append(gene)
len(common_genes)

7106 number of common genes
1442 number of unique genes
(24811, 15)


7106

In [45]:
# common_loops_normal
common_loops_normal = pd.read_csv('../data/processed/fig2_hichip/common_loops_normal.csv',index_col=0)
common_loops_normal[['loop_1','loop_2']] = common_loops_normal.loop_name.str.split('::',expand=True)
common_loops_normal.shape

(3717, 3)

In [47]:
print(common_loops_normal.merge(anc_prom[['name','thickEnd']], how='inner',left_on='loop_1',right_on='thickEnd').shape)
print(common_loops_normal.merge(anc_prom[['name','thickEnd']], how='inner',left_on='loop_2',right_on='thickEnd').shape)


# basically saing

(325, 5)
(496, 5)


In [28]:
# common_genes_in_common_loops
common_genes_in_common_loops = set()
for tissue in normal_tissues:
    common_loops_normal['loop_1_'+tissue] = ''
    common_loops_normal['loop_2_'+tissue] = ''
    anc_prom = pybedtools.BedTool('../data/interim/annon/promoter_anchors/promoter_'+normal_tissues[1]+'_annon.bed').to_dataframe()
    anc_prom = anc_prom[anc_prom.name.isin(common_genes)]
    genes1 = common_loops_normal.merge(anc_prom[['name','thickEnd']], how='inner',left_on='loop_1',right_on='thickEnd').name.unique()#.sort()
    genes1 = sorted(genes1)
    genes2 = common_loops_normal.merge(anc_prom[['name','thickEnd']], how='inner',left_on='loop_2',right_on='thickEnd').name.unique()#.sort()
    genes2 = sorted(genes2)
    genes = sorted(set(genes1).union(set(genes2)))
    print(tissue)
    common_genes_in_common_loops= common_genes_in_common_loops.union(genes)
    print(len(common_genes_in_common_loops))
#     print(genes)


Airway
453
Astrocytes
453
Bladder
453
Colon
453
Esophageal
453
GDSD6
453
GM12878
453
HMEC
453
Melanocytes
453
Ovarian
453
Pancreas
453
Prostate
453
Renal
453
Thyroid
453
Uterine
453


In [40]:
# housekeeping and essential genes
housekeeping_df = pd.read_table('../data/external/HSIAO_HOUSEKEEPING_GENES.txt',skiprows=2, header=None)
housekeeping_df.columns = ['genes']
housekeeping_genes_list = list(housekeeping_df.genes)
print('housekeeping_genes_list', len(housekeeping_genes_list))

essential_genes = pd.read_csv('../data/external/essential_genes/essential_genes_wang2015.txt',header=None).iloc[:,0].to_list()
print('essential_genes_list', len(essential_genes))


housekeeping_genes_list 398
essential_genes_list 1878


In [44]:
num_hk_cg_cl = len(set(common_genes_in_common_loops).intersection(set(housekeeping_genes_list)))
num_es_cg_cl = len(set(common_genes_in_common_loops).intersection(set(essential_genes)))
print('num housekeeping genes in commongenes-common loops', num_hk_cg_cl)
print('num essential genes in commongenes-common loops', num_es_cg_cl)

num housekeeping genes in commongenes-common loops 36
num essential genes in commongenes-common loops 112


In [52]:
# hk genes
print(sorted(set(common_genes_in_common_loops).intersection(set(housekeeping_genes_list))))

['ACTB', 'CAPNS1', 'CIRBP', 'COX6A1', 'CTDSP2', 'DDB1', 'DHPS', 'EEF1G', 'EIF4G2', 'EZR', 'FBL', 'FTL', 'GPX4', 'HINT1', 'HNRNPF', 'HSPB1', 'IFITM3', 'JUNB', 'JUND', 'MLF2', 'NACA', 'PFN1', 'PHB2', 'PSAP', 'PTP4A2', 'RPL10A', 'RPL23', 'RPL3', 'RPL35', 'RPS10', 'RPS11', 'RPS21', 'RPS7', 'STMN1', 'TCEA1', 'YWHAZ']


In [51]:
# es genes
print(sorted(set(common_genes_in_common_loops).intersection(set(essential_genes))))

['ACO2', 'ACTB', 'ACTR3', 'AHCY', 'ANAPC15', 'AP2S1', 'ARID1A', 'BCR', 'CCT2', 'CENPM', 'CHAF1A', 'CHEK1', 'CLP1', 'COPZ1', 'COQ5', 'CREB3', 'DCTN3', 'DDB1', 'DDX27', 'DHX8', 'DPAGT1', 'ECD', 'EEF1G', 'EIF4G2', 'ELP5', 'ELP6', 'EP300', 'FPGS', 'GET4', 'GRB2', 'GSS', 'GTPBP8', 'HEXIM1', 'INTS1', 'IPO13', 'IRF2BP2', 'KAT7', 'KRAS', 'MCL1', 'METTL3', 'MNAT1', 'MRPL16', 'MRPL24', 'MRPL38', 'MRPL48', 'MRPS21', 'NAA15', 'NDUFA2', 'NOP9', 'NPLOC4', 'NSMCE1', 'NUBP1', 'NXF1', 'PCBP1', 'PFN1', 'PHB2', 'PPA1', 'PPIL2', 'PPP1CB', 'PPP4C', 'PRCC', 'PRELID1', 'PRMT1', 'PSMD4', 'RAD9A', 'REXO2', 'RNASEH2A', 'RNASEH2C', 'RPL10A', 'RPL23', 'RPL3', 'RPL35', 'RPP14', 'RPS10', 'RPS11', 'RPS21', 'RPS7', 'RTTN', 'SAMD4B', 'SCD', 'SCYL1', 'SEPHS2', 'SHMT2', 'SIK2', 'SLC35B1', 'SLC3A2', 'SMC3', 'SMG9', 'SNAPC1', 'SRP54', 'SRRT', 'SRSF1', 'SSBP4', 'STK11', 'TAF10', 'TAF15', 'TAF8', 'TCEA1', 'TERF2', 'TPX2', 'TRIAP1', 'TRPM7', 'TXN2', 'UBE2M', 'UBTF', 'UQCC2', 'WDR55', 'WDR74', 'WEE1', 'ZFC3H1', 'ZFP36L1', 'ZM